from google.colab import drive

# Accessing My Google Drive
drive.mount('/content/drive')
datapath="drive/My Drive/MLVC_competition"

!unzip '/content/drive/MyDrive/MLVC_competition/dataset.zip'

# Define Model

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class LeNet_STL10(nn.Module):
    def __init__(self):
        super(LeNet_STL10, self).__init__()
        self.num_classes = 10

        self.relu = nn.ReLU(inplace=True)
        self.conv1 = nn.Conv2d(3, 6, kernel_size=5, stride=1)
        self.pool1 = nn.MaxPool2d((2, 2), stride=2)
        self.conv2 = nn.Conv2d(6, 16, kernel_size=5, stride=1)
        self.pool2 = nn.MaxPool2d((2, 2), stride=2)
        self.fc1 = nn.Linear(400, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, self.num_classes)

        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')

    def forward(self, x):
        x = self.relu(self.conv1(x))
        x = self.pool1(x)

        x = self.relu(self.conv2(x))
        x = self.pool2(x)
        x = torch.flatten(x, 1)

        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))
        x = self.fc3(x)

        return x


def Model():
    """Return your custom model
    """
    return LeNet_STL10()

In [2]:
'''ResNet in PyTorch.
For Pre-activation ResNet, see 'preact_resnet.py'.
Reference:
[1] Kaiming He, Xiangyu Zhang, Shaoqing Ren, Jian Sun
    Deep Residual Learning for Image Recognition. arXiv:1512.03385
'''



class BasicBlock(nn.Module):
    expansion = 1

    def __init__(self, in_planes, planes, stride=1):
        super(BasicBlock, self).__init__()
        self.conv1 = nn.Conv2d(
            in_planes, planes, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3,
                               stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes)

        self.shortcut = nn.Sequential()
        if stride != 1 or in_planes != self.expansion*planes:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_planes, self.expansion*planes,
                          kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(self.expansion*planes)
            )

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        out += self.shortcut(x)
        out = F.relu(out)
        return out


class Bottleneck(nn.Module):
    expansion = 4

    def __init__(self, in_planes, planes, stride=1):
        super(Bottleneck, self).__init__()
        self.conv1 = nn.Conv2d(in_planes, planes, kernel_size=1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3,
                               stride=stride, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes)
        self.conv3 = nn.Conv2d(planes, self.expansion *
                               planes, kernel_size=1, bias=False)
        self.bn3 = nn.BatchNorm2d(self.expansion*planes)

        self.shortcut = nn.Sequential()
        if stride != 1 or in_planes != self.expansion*planes:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_planes, self.expansion*planes,
                          kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(self.expansion*planes)
            )

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = F.relu(self.bn2(self.conv2(out)))
        out = self.bn3(self.conv3(out))
        out += self.shortcut(x)
        out = F.relu(out)
        return out


class ResNet(nn.Module):
    def __init__(self, block, num_blocks, num_classes=10):
        super(ResNet, self).__init__()
        self.in_planes = 64

        self.conv1 = nn.Conv2d(3, 64, kernel_size=3,
                               stride=1, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.layer1 = self._make_layer(block, 64, num_blocks[0], stride=1)
        self.layer2 = self._make_layer(block, 128, num_blocks[1], stride=2)
        self.layer3 = self._make_layer(block, 256, num_blocks[2], stride=2)
        #self.layer4 = self._make_layer(block, 512, num_blocks[3], stride=2)
        self.linear = nn.Linear(1024, num_classes)
        #self.linear = nn.Linear(512*block.expansion, num_classes)

    def _make_layer(self, block, planes, num_blocks, stride):
        strides = [stride] + [1]*(num_blocks-1)
        layers = []
        for stride in strides:
            layers.append(block(self.in_planes, planes, stride))
            self.in_planes = planes * block.expansion
        return nn.Sequential(*layers)

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        #out = self.layer4(out)
        out = F.avg_pool2d(out, 4)
        out = out.view(out.size(0), -1)
        #print(out.size())
        out = self.linear(out)
        return out


def ResNet18():
    return ResNet(BasicBlock, [2, 2, 1])


def ResNet34():
    return ResNet(BasicBlock, [3, 4, 6, 3])


def ResNet50():
    return ResNet(Bottleneck, [3, 4, 6, 3])


def ResNet101():
    return ResNet(Bottleneck, [3, 4, 23, 3])


def ResNet152():
    return ResNet(Bottleneck, [3, 8, 36, 3])


def test():
    net = ResNet18()
    y = net(torch.randn(1, 3, 32, 32))
    print(y.size())

# test()

In [3]:
model=ResNet18()
print(model)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (shortcut): Sequential()
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=

In [4]:
pytorch_total_params = sum(p.numel() for p in model.parameters())
print(pytorch_total_params)

1604682


# Utils

In [5]:
class AverageMeter(object):
    r"""Computes and stores the average and current value
    """
    def __init__(self, name, fmt=':f'):
        self.name = name
        self.fmt = fmt
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

    def __str__(self):
        fmtstr = '{name} {val' + self.fmt + '} ({avg' + self.fmt + '})'
        return fmtstr.format(**self.__dict__)


class ProgressMeter(object):
    def __init__(self, num_batches, *meters, prefix=""):
        self.batch_fmtstr = self._get_batch_fmtstr(num_batches)
        self.meters = meters
        self.prefix = prefix

    def print(self, batch):
        entries = [self.prefix + self.batch_fmtstr.format(batch)]
        entries += [str(meter) for meter in self.meters]
        print('\t'.join(entries))

    def _get_batch_fmtstr(self, num_batches):
        num_digits = len(str(num_batches // 1))
        fmt = '{:' + str(num_digits) + 'd}'
        return '[' + fmt + '/' + fmt.format(num_batches) + ']'


def accuracy(output, target, topk=(1,)):
    r"""Computes the accuracy over the $k$ top predictions for the specified values of k
    """
    with torch.no_grad():
        maxk = max(topk)
        batch_size = target.size(0)

        # _, pred = output.topk(maxk, 1, True, True)
        # pred = pred.t()
        # correct = pred.eq(target.view(1, -1).expand_as(pred))

        # faster topk (ref: https://github.com/pytorch/pytorch/issues/22812)
        _, idx = output.sort(descending=True)
        pred = idx[:,:maxk]
        pred = pred.t()
        correct = pred.eq(target.view(1, -1).expand_as(pred))

        res = []

        for k in topk:
            correct_k = correct[:k].reshape(-1).float().sum(0, keepdim=True)
            res.append(correct_k.mul_(100.0 / batch_size))
        return res

#Hyperparameter

In [6]:
SAVEPATH = './weight/1129resnet/'
WEIGHTDECAY = 5e-4
MOMENTUM = 0.9
BATCHSIZE = 256
LR = 0.1
EPOCHS = 1000
PRINTFREQ = 10

#Train Model

In [ ]:
import time

import torch
import torch.nn as nn

import torchvision
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader


def main():
   model = ResNet18()

   ##### optimizer / learning rate scheduler / criterion #####
   optimizer = torch.optim.SGD(model.parameters(), lr=LR,
                               momentum=MOMENTUM, weight_decay=WEIGHTDECAY,
                               nesterov=True)
   scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer, [100, 150],
                                                    gamma=0.1)
   criterion = torch.nn.CrossEntropyLoss()
   ###########################################################

   model = model.cuda()
   criterion = criterion.cuda()

   # Check number of parameters your model
   pytorch_total_params = sum(p.numel() for p in model.parameters())
   print(f"Number of parameters: {pytorch_total_params}")
   
   if int(pytorch_total_params) > 2000000:
       print('Your model has the number of parameters more than 2 millions..')
       return
   
   normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                    std=[0.229, 0.224, 0.225])
   train_transform = transforms.Compose([
       transforms.RandomCrop(32, padding=4),
       transforms.RandomHorizontalFlip(),
       transforms.ToTensor(),
       normalize
   ])

   valid_transform = transforms.Compose([
       transforms.ToTensor(),
       normalize
   ])

   train_dataset = torchvision.datasets.ImageFolder(
       '/home/dhk1349/Desktop/Machine Learning/train', transform=train_transform)
   train_loader = DataLoader(train_dataset,
                             batch_size=BATCHSIZE, shuffle=True,
                             num_workers=4, pin_memory=True)
   
   val_dataset = torchvision.datasets.ImageFolder('/home/dhk1349/Desktop/Machine Learning/valid', transform=valid_transform)
   val_loader = DataLoader(val_dataset, batch_size=BATCHSIZE, shuffle=True, num_workers=4, pin_memory=True)

   last_top1_acc = 0

   # LOSS FOR EARLY STOPPING
   best_valid_loss = float('inf')
   early_stop_check = 0

   for epoch in range(EPOCHS):
       print("\n----- epoch: {}, lr: {} -----".format(
           epoch, optimizer.param_groups[0]["lr"]))

       # train for one epoch
       start_time = time.time()
       last_top1_acc, train_loss, train_acc = train(train_loader, epoch, model, optimizer, criterion)
       elapsed_time = time.time() - start_time
       print('==> {:.2f} seconds to train this epoch\n'.format(
           elapsed_time))
       
       # LOSS FOR EARLY STOPPING
       valid_loss, valid_acc = valid(val_loader, epoch, model, optimizer, criterion)
       """
       if early_stop_check == PATIENCE:
         print('\n******* Early Stopping *******\n')
         break
       """
       if valid_loss < best_valid_loss:
         early_stop_check = 0
         best_valid_loss = valid_loss
         torch.save(model.state_dict(), SAVEPATH+'earlystop_check_best.pth')
         print(f'\n\t## SAVE valid_loss: {valid_loss:.3f} | valid_acc: {valid_acc:.3f} ##')
       else:
         early_stop_check += 1

       # learning rate scheduling
       scheduler.step()

       # Save model each epoch
       torch.save(model.state_dict(), SAVEPATH+'earlystop_check.pth')

       print(f'\n\t==Epoch: {epoch + 1:02} | Epoch Time: {elapsed_time:.2f}s==')
       print(f'\t==Train Loss: {train_loss:.3f} | Train acc: {train_acc:.3f}==')
       print(f'\t==Valid Loss: {valid_loss:.3f} | Valid acc: {valid_acc:.3f}==\n')

   print(f"Last Top-1 Accuracy: {last_top1_acc}")
   print(f"Number of parameters: {pytorch_total_params}")


def train(train_loader, epoch, model, optimizer, criterion):
   batch_time = AverageMeter('Time', ':6.3f')
   data_time = AverageMeter('Data', ':6.3f')
   losses = AverageMeter('Loss', ':.4e')
   top1 = AverageMeter('Acc@1', ':6.2f')
   top5 = AverageMeter('Acc@5', ':6.2f')
   progress = ProgressMeter(len(train_loader), batch_time, data_time, losses,
                            top1, top5, prefix="Epoch: [{}]".format(epoch))
   
   # LOSS FOR EARLYSTOPPPING
   total_loss = 0
   iter_num = 0
   train_acc = 0

   # switch to train mode
   model.train()

   end = time.time()
   for i, (input, target) in enumerate(train_loader):
       # measure data loading time
       data_time.update(time.time() - end)

       input = input.cuda()
       target = target.cuda()

       # compute output
       output = model(input)
       loss = criterion(output, target)

       # measure accuracy and record loss, accuracy
       acc1, acc5 = accuracy(output, target, topk=(1, 5))
       losses.update(loss.item(), input.size(0))
       top1.update(acc1[0].item(), input.size(0))
       top5.update(acc5[0].item(), input.size(0))

       # compute gradient and do SGD step
       optimizer.zero_grad()
       loss.backward()
       optimizer.step()

       # LOSS FOR EARLYSTOPPING
       total_loss += loss.item()
       iter_num += 1
       train_acc += acc1.item()

       # measure elapsed time
       batch_time.update(time.time() - end)
       end = time.time()

       if i % PRINTFREQ == 0:
           progress.print(i)

   print('=> Acc@1 {top1.avg:.3f} Acc@5 {top5.avg:.3f}'
         .format(top1=top1, top5=top5))
   return top1.avg, total_loss / iter_num, train_acc / iter_num

def valid(val_loader, epoch, model, optimizer, criterion):
   # LOSS FOR EARLYSTOPPPING
   total_loss = 0
   iter_num = 0
   val_acc = 0

   with torch.no_grad():
     for i, (input, target) in enumerate(val_loader):
       input = input.cuda()
       target = target.cuda()

       output = model(input)
       loss = criterion(output, target)
       acc1 = accuracy(output, target, topk=(1,))
       
       total_loss += loss.item()
       iter_num += 1
       val_acc += acc1[0].item()

     return total_loss / iter_num, val_acc / iter_num


if __name__ == "__main__":
    main()

Number of parameters: 1604682

----- epoch: 0, lr: 0.1 -----
Epoch: [0][  0/352]	Time  0.814 ( 0.814)	Data  0.224 ( 0.224)	Loss 2.3370e+00 (2.3370e+00)	Acc@1  14.06 ( 14.06)	Acc@5  53.91 ( 53.91)
Epoch: [0][ 10/352]	Time  0.152 ( 0.214)	Data  0.000 ( 0.021)	Loss 2.2972e+00 (3.2436e+00)	Acc@1  16.80 ( 13.32)	Acc@5  63.28 ( 55.50)
Epoch: [0][ 20/352]	Time  0.152 ( 0.185)	Data  0.000 ( 0.011)	Loss 2.0467e+00 (2.7331e+00)	Acc@1  22.27 ( 16.29)	Acc@5  76.95 ( 62.97)
Epoch: [0][ 30/352]	Time  0.153 ( 0.175)	Data  0.000 ( 0.007)	Loss 2.0215e+00 (2.5122e+00)	Acc@1  24.22 ( 19.08)	Acc@5  78.12 ( 67.33)
Epoch: [0][ 40/352]	Time  0.159 ( 0.171)	Data  0.000 ( 0.006)	Loss 2.0132e+00 (2.3866e+00)	Acc@1  27.34 ( 20.60)	Acc@5  77.73 ( 70.20)
Epoch: [0][ 50/352]	Time  0.162 ( 0.169)	Data  0.000 ( 0.005)	Loss 1.9408e+00 (2.3034e+00)	Acc@1  27.73 ( 22.17)	Acc@5  83.59 ( 72.42)
Epoch: [0][ 60/352]	Time  0.156 ( 0.168)	Data  0.000 ( 0.004)	Loss 1.8449e+00 (2.2413e+00)	Acc@1  31.64 ( 23.13)	Acc@5  84.77 ( 7

Epoch: [1][230/352]	Time  0.156 ( 0.158)	Data  0.000 ( 0.001)	Loss 1.2761e+00 (1.3534e+00)	Acc@1  53.12 ( 50.39)	Acc@5  94.53 ( 93.68)
Epoch: [1][240/352]	Time  0.153 ( 0.158)	Data  0.000 ( 0.001)	Loss 1.3099e+00 (1.3507e+00)	Acc@1  53.52 ( 50.54)	Acc@5  92.97 ( 93.67)
Epoch: [1][250/352]	Time  0.156 ( 0.158)	Data  0.000 ( 0.001)	Loss 1.3046e+00 (1.3476e+00)	Acc@1  55.47 ( 50.68)	Acc@5  91.80 ( 93.69)
Epoch: [1][260/352]	Time  0.155 ( 0.158)	Data  0.000 ( 0.001)	Loss 1.2753e+00 (1.3439e+00)	Acc@1  53.52 ( 50.82)	Acc@5  94.53 ( 93.74)
Epoch: [1][270/352]	Time  0.155 ( 0.158)	Data  0.000 ( 0.001)	Loss 1.2603e+00 (1.3406e+00)	Acc@1  53.91 ( 50.96)	Acc@5  94.53 ( 93.76)
Epoch: [1][280/352]	Time  0.155 ( 0.157)	Data  0.000 ( 0.001)	Loss 1.3923e+00 (1.3370e+00)	Acc@1  50.78 ( 51.08)	Acc@5  91.80 ( 93.78)
Epoch: [1][290/352]	Time  0.155 ( 0.157)	Data  0.000 ( 0.001)	Loss 1.3309e+00 (1.3340e+00)	Acc@1  57.81 ( 51.21)	Acc@5  92.19 ( 93.82)
Epoch: [1][300/352]	Time  0.154 ( 0.157)	Data  0.000 ( 

Epoch: [3][ 80/352]	Time  0.155 ( 0.159)	Data  0.000 ( 0.003)	Loss 1.1077e+00 (1.0618e+00)	Acc@1  58.20 ( 61.64)	Acc@5  97.27 ( 96.37)
Epoch: [3][ 90/352]	Time  0.157 ( 0.159)	Data  0.000 ( 0.003)	Loss 1.0634e+00 (1.0621e+00)	Acc@1  61.33 ( 61.62)	Acc@5  95.70 ( 96.39)
Epoch: [3][100/352]	Time  0.156 ( 0.158)	Data  0.000 ( 0.002)	Loss 1.1412e+00 (1.0594e+00)	Acc@1  58.20 ( 61.80)	Acc@5  93.75 ( 96.41)
Epoch: [3][110/352]	Time  0.154 ( 0.158)	Data  0.000 ( 0.002)	Loss 1.1545e+00 (1.0591e+00)	Acc@1  56.64 ( 61.79)	Acc@5  96.09 ( 96.41)
Epoch: [3][120/352]	Time  0.156 ( 0.158)	Data  0.000 ( 0.002)	Loss 1.0300e+00 (1.0591e+00)	Acc@1  60.16 ( 61.79)	Acc@5  96.88 ( 96.42)
Epoch: [3][130/352]	Time  0.155 ( 0.158)	Data  0.000 ( 0.002)	Loss 1.0322e+00 (1.0601e+00)	Acc@1  64.84 ( 61.79)	Acc@5  95.70 ( 96.43)
Epoch: [3][140/352]	Time  0.155 ( 0.158)	Data  0.000 ( 0.002)	Loss 1.0488e+00 (1.0611e+00)	Acc@1  62.50 ( 61.72)	Acc@5  97.66 ( 96.45)
Epoch: [3][150/352]	Time  0.154 ( 0.157)	Data  0.000 ( 

Epoch: [4][310/352]	Time  0.154 ( 0.157)	Data  0.000 ( 0.001)	Loss 8.1556e-01 (9.4286e-01)	Acc@1  70.31 ( 66.42)	Acc@5  99.22 ( 97.08)
Epoch: [4][320/352]	Time  0.154 ( 0.157)	Data  0.000 ( 0.001)	Loss 9.6318e-01 (9.4287e-01)	Acc@1  67.19 ( 66.41)	Acc@5  96.09 ( 97.08)
Epoch: [4][330/352]	Time  0.155 ( 0.157)	Data  0.000 ( 0.001)	Loss 9.1690e-01 (9.4243e-01)	Acc@1  67.58 ( 66.41)	Acc@5  96.88 ( 97.08)
Epoch: [4][340/352]	Time  0.154 ( 0.157)	Data  0.000 ( 0.001)	Loss 9.0598e-01 (9.4178e-01)	Acc@1  65.23 ( 66.43)	Acc@5  97.27 ( 97.08)
Epoch: [4][350/352]	Time  0.155 ( 0.157)	Data  0.000 ( 0.001)	Loss 9.0321e-01 (9.4193e-01)	Acc@1  66.80 ( 66.41)	Acc@5  96.88 ( 97.07)
=> Acc@1 66.416 Acc@5 97.077
==> 55.16 seconds to train this epoch


	## SAVE valid_loss: 0.963 | valid_acc: 65.950 ##

	==Epoch: 05 | Epoch Time: 55.16s==
	==Train Loss: 0.942 | Train acc: 66.420==
	==Valid Loss: 0.963 | Valid acc: 65.950==


----- epoch: 5, lr: 0.1 -----
Epoch: [5][  0/352]	Time  0.412 ( 0.412)	Data  0.25

Epoch: [6][160/352]	Time  0.156 ( 0.157)	Data  0.000 ( 0.002)	Loss 8.8347e-01 (8.2338e-01)	Acc@1  72.66 ( 70.68)	Acc@5  98.44 ( 97.71)
Epoch: [6][170/352]	Time  0.155 ( 0.157)	Data  0.000 ( 0.002)	Loss 7.6462e-01 (8.2292e-01)	Acc@1  69.53 ( 70.69)	Acc@5  98.44 ( 97.72)
Epoch: [6][180/352]	Time  0.154 ( 0.157)	Data  0.000 ( 0.002)	Loss 7.6069e-01 (8.2130e-01)	Acc@1  74.61 ( 70.80)	Acc@5  98.83 ( 97.74)
Epoch: [6][190/352]	Time  0.156 ( 0.157)	Data  0.000 ( 0.001)	Loss 8.7739e-01 (8.2189e-01)	Acc@1  67.58 ( 70.74)	Acc@5  97.27 ( 97.76)
Epoch: [6][200/352]	Time  0.156 ( 0.157)	Data  0.000 ( 0.001)	Loss 8.2450e-01 (8.2181e-01)	Acc@1  69.92 ( 70.77)	Acc@5  96.88 ( 97.76)
Epoch: [6][210/352]	Time  0.154 ( 0.157)	Data  0.000 ( 0.001)	Loss 6.8402e-01 (8.2115e-01)	Acc@1  77.34 ( 70.80)	Acc@5  99.61 ( 97.77)
Epoch: [6][220/352]	Time  0.154 ( 0.157)	Data  0.000 ( 0.001)	Loss 7.0462e-01 (8.1855e-01)	Acc@1  78.52 ( 70.93)	Acc@5  98.05 ( 97.78)
Epoch: [6][230/352]	Time  0.155 ( 0.157)	Data  0.000 ( 

...


	==Epoch: 162 | Epoch Time: 54.87s==
	==Train Loss: 0.035 | Train acc: 99.217==
	==Valid Loss: 0.637 | Valid acc: 82.874==


----- epoch: 162, lr: 0.0010000000000000002 -----
Epoch: [162][  0/352]	Time  0.388 ( 0.388)	Data  0.229 ( 0.229)	Loss 2.2261e-02 (2.2261e-02)	Acc@1 100.00 (100.00)	Acc@5 100.00 (100.00)
Epoch: [162][ 10/352]	Time  0.154 ( 0.176)	Data  0.000 ( 0.021)	Loss 3.8063e-02 (3.1228e-02)	Acc@1  99.22 ( 99.40)	Acc@5 100.00 (100.00)
Epoch: [162][ 20/352]	Time  0.156 ( 0.166)	Data  0.000 ( 0.011)	Loss 3.7028e-02 (3.1987e-02)	Acc@1  99.22 ( 99.33)	Acc@5 100.00 (100.00)
Epoch: [162][ 30/352]	Time  0.154 ( 0.163)	Data  0.000 ( 0.008)	Loss 2.2456e-02 (3.3037e-02)	Acc@1  99.22 ( 99.31)	Acc@5 100.00 (100.00)
Epoch: [162][ 40/352]	Time  0.155 ( 0.161)	Data  0.000 ( 0.006)	Loss 2.9187e-02 (3.2106e-02)	Acc@1  99.61 ( 99.34)	Acc@5 100.00 (100.00)
Epoch: [162][ 50/352]	Time  0.155 ( 0.160)	Data  0.000 ( 0.005)	Loss 2.7888e-02 (3.1839e-02)	Acc@1  99.61 ( 99.35)	Acc@5 100.00 (100.00)
Ep

Epoch: [163][210/352]	Time  0.155 ( 0.156)	Data  0.000 ( 0.001)	Loss 3.8696e-02 (3.1581e-02)	Acc@1  99.22 ( 99.39)	Acc@5 100.00 (100.00)
Epoch: [163][220/352]	Time  0.155 ( 0.156)	Data  0.000 ( 0.001)	Loss 2.3057e-02 (3.1510e-02)	Acc@1 100.00 ( 99.40)	Acc@5 100.00 (100.00)
Epoch: [163][230/352]	Time  0.155 ( 0.156)	Data  0.000 ( 0.001)	Loss 3.2006e-02 (3.1531e-02)	Acc@1  99.61 ( 99.39)	Acc@5 100.00 (100.00)
Epoch: [163][240/352]	Time  0.156 ( 0.156)	Data  0.000 ( 0.001)	Loss 3.2460e-02 (3.1456e-02)	Acc@1  99.61 ( 99.40)	Acc@5 100.00 (100.00)
Epoch: [163][250/352]	Time  0.156 ( 0.156)	Data  0.000 ( 0.001)	Loss 3.2622e-02 (3.1448e-02)	Acc@1  99.61 ( 99.40)	Acc@5 100.00 (100.00)
Epoch: [163][260/352]	Time  0.154 ( 0.156)	Data  0.000 ( 0.001)	Loss 3.0238e-02 (3.1460e-02)	Acc@1  99.22 ( 99.40)	Acc@5 100.00 (100.00)
Epoch: [163][270/352]	Time  0.156 ( 0.156)	Data  0.000 ( 0.001)	Loss 4.5047e-02 (3.1628e-02)	Acc@1  99.22 ( 99.38)	Acc@5 100.00 (100.00)
Epoch: [163][280/352]	Time  0.155 ( 0.156

Epoch: [165][ 60/352]	Time  0.155 ( 0.159)	Data  0.000 ( 0.004)	Loss 3.1053e-02 (2.8540e-02)	Acc@1  98.83 ( 99.40)	Acc@5 100.00 (100.00)
Epoch: [165][ 70/352]	Time  0.156 ( 0.158)	Data  0.000 ( 0.003)	Loss 3.7148e-02 (2.8288e-02)	Acc@1  99.61 ( 99.44)	Acc@5 100.00 (100.00)
Epoch: [165][ 80/352]	Time  0.154 ( 0.158)	Data  0.000 ( 0.003)	Loss 1.9144e-02 (2.8082e-02)	Acc@1 100.00 ( 99.45)	Acc@5 100.00 (100.00)
Epoch: [165][ 90/352]	Time  0.156 ( 0.158)	Data  0.000 ( 0.003)	Loss 4.6331e-02 (2.8713e-02)	Acc@1  98.44 ( 99.41)	Acc@5 100.00 (100.00)
Epoch: [165][100/352]	Time  0.156 ( 0.157)	Data  0.000 ( 0.002)	Loss 2.3120e-02 (2.8719e-02)	Acc@1  99.61 ( 99.41)	Acc@5 100.00 (100.00)
Epoch: [165][110/352]	Time  0.155 ( 0.157)	Data  0.000 ( 0.002)	Loss 3.4003e-02 (2.9019e-02)	Acc@1  99.61 ( 99.40)	Acc@5 100.00 (100.00)
Epoch: [165][120/352]	Time  0.154 ( 0.157)	Data  0.000 ( 0.002)	Loss 4.7545e-02 (2.8667e-02)	Acc@1  98.44 ( 99.42)	Acc@5 100.00 (100.00)
Epoch: [165][130/352]	Time  0.156 ( 0.157

Epoch: [166][290/352]	Time  0.156 ( 0.156)	Data  0.000 ( 0.001)	Loss 3.5442e-02 (2.8170e-02)	Acc@1  98.83 ( 99.47)	Acc@5 100.00 (100.00)
Epoch: [166][300/352]	Time  0.155 ( 0.156)	Data  0.000 ( 0.001)	Loss 2.4468e-02 (2.8205e-02)	Acc@1  99.61 ( 99.47)	Acc@5 100.00 (100.00)
Epoch: [166][310/352]	Time  0.154 ( 0.156)	Data  0.000 ( 0.001)	Loss 6.0153e-02 (2.8386e-02)	Acc@1  98.05 ( 99.46)	Acc@5 100.00 (100.00)
Epoch: [166][320/352]	Time  0.156 ( 0.156)	Data  0.000 ( 0.001)	Loss 3.0708e-02 (2.8469e-02)	Acc@1  98.83 ( 99.46)	Acc@5 100.00 (100.00)
Epoch: [166][330/352]	Time  0.155 ( 0.156)	Data  0.000 ( 0.001)	Loss 3.0881e-02 (2.8561e-02)	Acc@1 100.00 ( 99.46)	Acc@5 100.00 (100.00)
Epoch: [166][340/352]	Time  0.156 ( 0.156)	Data  0.000 ( 0.001)	Loss 3.3453e-02 (2.8620e-02)	Acc@1  99.22 ( 99.45)	Acc@5 100.00 (100.00)
Epoch: [166][350/352]	Time  0.156 ( 0.156)	Data  0.000 ( 0.001)	Loss 2.4726e-02 (2.8633e-02)	Acc@1  99.22 ( 99.45)	Acc@5 100.00 (100.00)
=> Acc@1 99.450 Acc@5 100.000
==> 54.91 s

Epoch: [168][140/352]	Time  0.154 ( 0.157)	Data  0.000 ( 0.002)	Loss 3.0442e-02 (2.6971e-02)	Acc@1 100.00 ( 99.49)	Acc@5 100.00 (100.00)
Epoch: [168][150/352]	Time  0.154 ( 0.157)	Data  0.000 ( 0.002)	Loss 3.6208e-02 (2.7192e-02)	Acc@1  99.22 ( 99.50)	Acc@5 100.00 (100.00)
Epoch: [168][160/352]	Time  0.154 ( 0.157)	Data  0.000 ( 0.002)	Loss 3.1034e-02 (2.7008e-02)	Acc@1  99.22 ( 99.50)	Acc@5 100.00 (100.00)
Epoch: [168][170/352]	Time  0.160 ( 0.157)	Data  0.000 ( 0.002)	Loss 2.6629e-02 (2.6747e-02)	Acc@1  99.22 ( 99.50)	Acc@5 100.00 (100.00)
Epoch: [168][180/352]	Time  0.155 ( 0.156)	Data  0.000 ( 0.001)	Loss 2.6635e-02 (2.6934e-02)	Acc@1  99.61 ( 99.49)	Acc@5 100.00 (100.00)
Epoch: [168][190/352]	Time  0.158 ( 0.156)	Data  0.000 ( 0.001)	Loss 2.9097e-02 (2.7243e-02)	Acc@1  99.61 ( 99.48)	Acc@5 100.00 (100.00)
Epoch: [168][200/352]	Time  0.155 ( 0.156)	Data  0.000 ( 0.001)	Loss 3.1147e-02 (2.7340e-02)	Acc@1  99.22 ( 99.48)	Acc@5 100.00 (100.00)
Epoch: [168][210/352]	Time  0.156 ( 0.156